# Comparison of Explainers : GNNExplainer, EdgeSHAPer, regSHAP

In [2]:
# Imports
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../../EdgeSHAPer/src/')
sys.path.insert(0,'../../models/pcqm4m-v2_ogb/')
sys.path.insert(0,'../../utils/')
sys.path.insert(0,'../../../AMLD-2021-Graphs/src/')

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interact
from tkinter import *
matplotlib.use('TkAgg')

# PyTorch related
import torch
import torch.nn.functional as F
from torch_geometric.data.data import Data
from torch_geometric.utils import to_undirected
from torch_geometric.explain import Explainer,GNNExplainer,PGExplainer
from torch_geometric.explain.metric import fidelity
from gnn import GNN
from visualization import * # AMDL-2021-Graphs
from torch_geometric.utils import to_networkx
import networkx as nx

from utils import *

from regSHAPer import regSHAP
from edgeshaper import edgeshaper
from scipy.special import binom
from itertools import combinations

# Dataset-related
import ogb
from ogb.lsc import PCQM4Mv2Dataset, PygPCQM4Mv2Dataset
from ogb.utils import smiles2graph

# Chemistry related
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# ================== #
# Importing datasets #
# ================== #
path_data = "../../data/"
dataset_smiles = PCQM4Mv2Dataset(root=path_data,only_smiles = True)
dataset_PyG = PygPCQM4Mv2Dataset(root=path_data)
suppl = Chem.SDMolSupplier(path_data+'pcqm4m-v2-train.sdf')
n_train = 3378606 # Number of molecules in the train subset of pcqm4m-v2 dataset

In [4]:
# =================== #
# Importing the model #
# =================== #
path_model = "../../models/pcqm4m-v2_ogb/"
model_pt = torch.load(path_model+"model_trained.pt",map_location=torch.device('cpu'))

shared_params = {
        'num_layers': 5,
        'emb_dim': 600,
        'drop_ratio': 0,
        'graph_pooling': 'sum'
    }
device = torch.device("cuda:" + str(args.device)) if torch.cuda.is_available() else torch.device("cpu")
print(device)
model = GNN(gnn_type = 'gcn', virtual_node = False, **shared_params).to('cpu')
model.load_state_dict(model_pt['model_state_dict'])
epoch = model_pt['epoch']
print(epoch)

cpu
75


In [5]:
# ====================== #
# Importing explanations #
# ====================== #
path_script = "scripts/"
algo_files = {"GNNExplainer":"gnnexplainer/model_5gr_gnnexpl_dict.pt",
            "EdgeSHAPer":"EdgeSHAPer/5gr_edgeSHAPexpl_dict.pt",
            "regSHAPer":"regSHAPer/diff_5gr_regSHAP_dict.pt"}
algorithms = list(algo_files.keys())

algo_dicts = {}
algo_mol_indices = {}
for alg in algorithms:
    algo_dicts[alg] = torch.load(path_script+algo_files[alg])
    print(len(algo_dicts[alg].keys()), "molecules for "+alg)
    algo_mol_indices[alg] = list(algo_dicts[alg].keys())

42929 molecules for GNNExplainer
8100 molecules for EdgeSHAPer
11775 molecules for regSHAPer


In [6]:
# ===================== #
# Creating GNNExplainer #
# ===================== #
gnn_explainer = Explainer(
        model=model,
        algorithm=GNNExplainer(epochs=500),
        explanation_type="model",
        node_mask_type=None,
        edge_mask_type="object",
        model_config=dict(
            mode='regression',
            task_level='graph',
            return_type='raw',
        )
    )

## Metrics and visualization

### Metrics

#### FID +/-

In [6]:
idx_expl = np.arange(0,5000,1)
print("number of mols :",len(idx_expl))
idx_mols = np.array([algo_mol_indices["EdgeSHAPer"][i] for i in idx_expl])
list_graphs = dataset_PyG[idx_mols]
positive = False
for alg in algorithms:
    explanations = [torch.abs(torch.tensor(algo_dicts[alg][i])) for i in idx_mols]
    print(explanations[0])
    print(compute_fid(model,list_graphs,positive,explanations,5,None))

number of mols : 5000
tensor([0.0123, 0.9764, 0.9788, 0.9806, 0.0135, 0.9693, 0.9879, 0.9849, 0.9717,
        0.0122, 0.9740, 0.9762, 0.9717, 0.0842, 0.9786, 0.9835, 0.0151, 0.0114,
        0.0182, 0.0159, 0.0145, 0.0184, 0.0123, 0.0134, 0.0125, 0.0196, 0.9823,
        0.9724])
top-k 5


/var/folders/83/jd8c7w3s1p71rrnmtg0jd57w0000gn/T/ipykernel_20888/839069268.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  explanations = [torch.abs(torch.tensor(algo_dicts[alg][i])) for i in idx_mols]


(0.13126346272422962, 0.12505835782113012)
tensor([0.0052, 0.0399, 0.0022, 0.1177, 0.1430, 0.2785, 0.2788, 0.0370, 0.5538,
        0.1996, 0.0658, 0.0411, 0.2846, 0.1403, 0.2365, 0.0371, 0.0333, 0.0135,
        0.0178, 0.0581, 0.0043, 0.0078, 0.0108, 0.0381, 0.1350, 0.0399, 0.1013,
        0.0805], dtype=torch.float64)
top-k 5
(0.1310265111841072, 0.09681812852029278)
tensor([0.0466, 0.0656, 0.0675, 0.0285, 0.1390, 0.1062, 0.3056, 0.0676, 0.0683,
        0.0415, 0.0214, 0.0844, 0.0127, 0.0523], dtype=torch.float64)
top-k 5
(0.09989096711973033, 0.07042292326653166)


#### GEF

In [7]:
# Creating list of graphs and explanations
idx_mols = algo_mol_indices[alg][:5000]
list_graphs = dataset_PyG[idx_mols]

expl = [torch.tensor(algo_dicts[alg][i]) for i in idx_mols]
k = 5
dict_gef = compute_gef(idx_mols,list_graphs,algo_dicts,model,k,algorithms)

/Users/evandieren/Desktop/epfl/Q2/GNN_Explainability/experiments/Comparison_Explainers/../../utils/utils.py:305: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  expl = [torch.tensor(np.abs(algo_dicts[alg][i])) for i in idx_mols]
100%|██████████████████████████████████████| 5000/5000 [00:45<00:00, 109.92it/s]
/Users/evandieren/Desktop/epfl/Q2/GNN_Explainability/experiments/Comparison_Explainers/../../utils/utils.py:266: RuntimeWarning: divide by zero encountered in divide
  D_KL = lambda p,q: np.sum(p*np.where(p*q != 0, np.log(p/q), 0))
/Users/evandieren/Desktop/epfl/Q2/GNN_Explainability/experiments/Comparison_Explainers/../../utils/utils.py:266: RuntimeWarning: invalid value encountered in divide
  D_KL = lambda p,q: np.sum(p*np.where(p*q != 0, np.log(p/q), 0))
/Users/evandieren/Desktop/epfl/Q2/GNN_Explainability/experiments/Comparison_E

GEF GNNExplainer :  0.4019372073618389


100%|██████████████████████████████████████| 5000/5000 [00:43<00:00, 114.30it/s]


GEF EdgeSHAPer :  0.3619845126284843


100%|██████████████████████████████████████| 5000/5000 [00:41<00:00, 119.51it/s]

GEF regSHAPer :  0.26564437405896824


In [10]:
print("GNNExplainer:",dict_gef["GNNExplainer"][1])
print("EdgeSHAPer:",dict_gef["EdgeSHAPer"][1])
print("regSHAPer:",dict_gef["regSHAPer"][1])
# regSHAP is better for this metric. (the lower the better)

GNNExplainer: 0.4019372073618389
EdgeSHAPer: 0.3619845126284843
regSHAPer: 0.26564437405896824


In [11]:
# verif mean
y = torch.tensor([data.y for data in list_graphs])
print(torch.mean(y))
print(np.mean(dict_gef["GNNExplainer"][0][:,1]))

tensor(6.9843)
6.6923107493400575


### Plots for the report

#### Graph vs Subgraph predictions

In [7]:
# Showing graph vs subgraph
col_dict = {"GNNExplainer":"red",
            "EdgeSHAPer":"mediumblue",
            "regSHAPer":"orange"}
# Creating list of graphs and explanations
idx_mols = algo_mol_indices[alg][:5000]
list_graphs = dataset_PyG[idx_mols]

In [8]:
for k in [3,5,7,9]:
    dict_gef = compute_gef(idx_mols,list_graphs,algo_dicts,model,k,algorithms)
    graph_subgraph(dict_gef,k,col_dict,(2,14),(2,14),"plots/pred_graph_subgraph/graph_subgraph_k_"+str(k)+".png")

/Users/evandieren/Desktop/epfl/Q2/GNN_Explainability/experiments/Comparison_Explainers/../../utils/utils.py:305: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  expl = [torch.tensor(np.abs(algo_dicts[alg][i])) for i in idx_mols]
100%|███████████████████████████████████████| 5000/5000 [00:56<00:00, 87.90it/s]
/Users/evandieren/Desktop/epfl/Q2/GNN_Explainability/experiments/Comparison_Explainers/../../utils/utils.py:266: RuntimeWarning: divide by zero encountered in divide
  D_KL = lambda p,q: np.sum(p*np.where(p*q != 0, np.log(p/q), 0))
/Users/evandieren/Desktop/epfl/Q2/GNN_Explainability/experiments/Comparison_Explainers/../../utils/utils.py:266: RuntimeWarning: invalid value encountered in divide
  D_KL = lambda p,q: np.sum(p*np.where(p*q != 0, np.log(p/q), 0))
/Users/evandieren/Desktop/epfl/Q2/GNN_Explainability/experiments/Comparison_E

GEF GNNExplainer :  0.4355784716115939


100%|██████████████████████████████████████| 5000/5000 [00:48<00:00, 102.55it/s]


GEF EdgeSHAPer :  0.4252967821093995


100%|██████████████████████████████████████| 5000/5000 [00:45<00:00, 110.48it/s]


...

k is bigger than the number of bonds !
k is bigger than the number of bonds !


100%|█████████████████████████████████████▉| 4994/5000 [00:52<00:00, 114.71it/s]

k is bigger than the number of bonds !
k is bigger than the number of bonds !
k is bigger than the number of bonds !


100%|███████████████████████████████████████| 5000/5000 [00:52<00:00, 94.54it/s]


GEF regSHAPer :  0.10868193277707772


In [9]:
# MAE for last k (verif)
for i,alg in enumerate(dict_gef.keys()):
    print(alg)
    print(np.mean((dict_gef[alg][0][:,0]-dict_gef[alg][0][:,1])**2))

GNNExplainer
0.6967954875003494
EdgeSHAPer
0.46080155783267873
regSHAPer
0.25969855315680995


#### Threshold graph

In [7]:
thresh = np.arange(0,1.01,0.01)
compute_tresh(thresh,5000,algo_dicts,col_dict,algorithms,"plots/threshold/threshold_plot.png")

GNNExplainer


/Users/evandieren/Desktop/epfl/Q2/GNN_Explainability/experiments/Comparison_Explainers/../../utils/utils.py:376: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  expl = [refactor(torch.tensor(np.abs(algo_dicts[alg][i])),
100%|██████████████████████████████████████| 5000/5000 [00:15<00:00, 327.60it/s]


# expl :  5000
EdgeSHAPer


100%|██████████████████████████████████████| 5000/5000 [00:12<00:00, 385.42it/s]


# expl :  5000
regSHAPer


100%|██████████████████████████████████████| 5000/5000 [00:14<00:00, 342.12it/s]


# expl :  5000


#### Molecule visualization

In [17]:
idx_mols = [629596,634384]

mol_viz(algorithms,idx_mols,(10,10),dataset_PyG,algo_dicts,"./plots/expl_viz/expl_mol"+str(idx_mols))

15
done
15
done
15
done
12
done
12
done
12
done


In [16]:
idx_mols = [algo_mol_indices[alg][idx] for idx in np.random.randint(5000,size=4)]
idx_mols = [[idx_mols[0],idx_mols[1]],[idx_mols[2],idx_mols[3]]]
for i in idx_mols:
    mol_viz(algorithms,i,(10,10),dataset_PyG,algo_dicts,"./plots/expl_viz/expl_mol"+str(i))

/Users/evandieren/Desktop/epfl/Q2/GNN_Explainability/experiments/Comparison_Explainers/../../utils/utils.py:517: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  expl = refactor(torch.tensor(np.abs(algo_dicts[alg][idx_mol])),False,False)


14
done
14
done
14
done
9
done
9
done
9
done
16
done
16
done
16
done
15
done
15
done
15
done


# Other snippets of code and/or old code

In [ ]:

idx_random = np.random.randint(5)
idx_random

In [ ]:
# Show explanation -> broken due to indices missmatch
idx = algo_mol_indices["regSHAP"][40]
print(idx)
show_explanation(algorithms,dataset_PyG,dataset_smiles,idx,algo_dicts,"jet")

In [ ]:
# typical barplot instead
idx = algo_mol_indices["regSHAP"][40]
show_barplot_explanation(algorithms,dataset_PyG,idx,algo_dicts,"out.png")

In [ ]:
# Verifying the efficiency Shapley property
data = dataset_PyG[0]
x = data.x
y = data.y
e1 = data.edge_index[:,4:16]
e2 = data.edge_index[:,-2:]

edge_index = torch.cat((e1,e2),axis=1)
new_edge_attr = torch.cat((data.edge_attr[4:16,:],data.edge_attr[-2:,:]),axis=0)
new_x_idx = np.unique(edge_index[0]) # selecting the nodes which degree is > 0
new_x = data.x[new_x_idx,:]
new_edge_index = torch.vstack((rank_arr(edge_index[0],False),rank_arr(edge_index[1],False)))
new_graph = Data(x=new_x,edge_index=new_edge_index,edge_attr=new_edge_attr,y=y)
test_regSHAP = regSHAP(new_graph,64,model,42,False,False)
batch = torch.zeros(new_graph.x.shape[0], dtype=int, device=new_graph.x.device)
print(abs(sum(test_regSHAP)))
print(model(new_graph.x,new_graph.edge_index,new_graph.edge_attr,batch))

In [ ]:
# Computing the mean, min and max number of edges in the 5000 molecules
mol_idx = algo_mol_indices[alg][:5000]
c = 0
c_min = 50
c_max = 0
for mol in mol_idx:
    explanation = algo_dicts["regSHAP"][mol]
    c_min = min(c_min,len(explanation))
    c_max = max(c_max,len(explanation))
    c += len(explanation)
print(c/5000)
print(c_min)
print(c_max)